<a href="https://colab.research.google.com/github/aarondelgiudice/thinkful_data_bootcamp/blob/master/unit_7/capstone/RD/7_1a_Data_Wrangling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Wrangling
- Audio Features
- Lyric Data

## Load Packages

In [0]:
# install spotiPy package
#! pip install spotipy

In [0]:
# install lyics genius package
#!pip install lyricsgenius

In [0]:
import numpy as np
import pandas as pd

#import spotiPy
#import spotipy.oauth2 as oauth2
#import lyric genius
#import lyricsgenius

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

### Import local data

In [0]:
#from google.colab import files
#files.upload()

In [0]:
filename = "spotify_test_data"
df_af = pd.read_csv(filename + ".csv", index_col=0)

df_af.head()

,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,target,tempo,time_signature,track_href,type,uri,valence,track_names,artist_names,album_names,release_dates,popularity,explicit
0,0.1730,https://api.spotify.com/v1/audio-analysis/7eEl...,0.796,197236,0.467,7eElVgPcxDqmgGrSwVFI74,0.004110,6,0.1260,-10.369,1,0.1810,1,107.901,4,https://api.spotify.com/v1/tracks/7eElVgPcxDqm...,audio_features,spotify:track:7eElVgPcxDqmgGrSwVFI74,0.516,Sober,Lorde,Melodrama,2017-06-16,59,True
1,0.2870,https://api.spotify.com/v1/audio-analysis/02oA...,0.771,189796,0.431,02oAUzv4M0ItuTDy2RT3IT,0.000003,10,0.0934,-5.423,1,0.0577,1,107.047,4,https://api.spotify.com/v1/tracks/02oAUzv4M0It...,audio_features,spotify:track:02oAUzv4M0ItuTDy2RT3IT,0.268,Homemade Dynamite,Lorde,Melodrama,2017-06-16,60,True
2,0.2390,https://api.spotify.com/v1/audio-analysis/5q4B...,0.664,271088,0.382,5q4BpnMrYEFzLO0dYODj6J,0.007490,0,0.0943,-9.977,1,0.0470,1,123.214,4,https://api.spotify.com/v1/tracks/5q4BpnMrYEFz...,audio_features,spotify:track:5q4BpnMrYEFzLO0dYODj6J,0.126,The Louvre,Lorde,Melodrama,2017-06-16,61,False
3,0.9200,https://api.spotify.com/v1/audio-analysis/6Kkt...,0.587,171728,0.229,6Kkt27YmFyIFrcX3QXFi2o,0.000000,10,0.1000,-11.254,0,0.1280,1,75.670,4,https://api.spotify.com/v1/tracks/6Kkt27YmFyIF...,audio_features,spotify:track:6Kkt27YmFyIFrcX3QXFi2o,0.379,Liability,Lorde,Melodrama,2017-06-16,69,False
4,0.0328,https://api.spotify.com/v1/audio-analysis/1Dp7...,0.487,367391,0.445,1Dp7JGFNjvg8Nk0CtMCcnr,0.010400,1,0.0778,-10.959,0,0.1130,1,97.031,4,https://api.spotify.com/v1/tracks/1Dp7JGFNjvg8...,audio_features,spotify:track:1Dp7JGFNjvg8Nk0CtMCcnr,0.180,Hard Feelings/Loveless,Lorde,Melodrama,2017-06-16,60,True


In [0]:
# change field size to avoid ParseError
import sys
import csv

csv.field_size_limit(sys.maxsize)

filename = "lyric_test_data_clean"
# set engine="python" to avoid ParseError
# set error_bad_lines=False
#df_lyrics = pd.read_csv(filename + ".csv", index_col=0,
#                        engine="python", error_bad_lines=False)
df_lyrics = pd.read_csv(filename + ".csv", index_col=0)
# make sure lyric data is type string
df_lyrics['lyrics_raw'] = df_lyrics['lyrics_raw'].astype(str)
df_lyrics['lyrics_clean'] = df_lyrics['lyrics_clean'].astype(str)
df_lyrics.head()

,target,artist_names,track_names,lyrics_raw,lyrics_clean
0,1,Lorde,Sober,"[Intro]\nNight, midnight, lose my mind\nNight,...",intro night midnight lose mind night midnight ...
1,1,Lorde,Homemade Dynamite,[Verse 1]\nA couple rebel top gun pilots\nFlyi...,verse couple rebel top gun pilot flying nowher...
2,1,Lorde,The Louvre,"[Verse 1]\nWell, summer slipped us underneath ...",verse well summer slipped u underneath tongue ...
3,1,Lorde,Liability,"[Intro]\nOne, two\n\n[Verse 1]\nBaby really hu...",intro one two verse baby really hurt cry taxi ...
4,1,Lorde,Hard Feelings/Loveless,Hard Feelings\n\n(Go back and tell it)\n\nPlea...,hard feeling go back tell please could tender ...


## Spotify Data

### Connect to API

To use the Spotify API first create an app within the Spotify developer environment.

Go to the Spotify Developer page, log in and navigate to the **My Applications** page.

Create a new application and specify a name. Make note of the `Client ID` and `Client Secret`, they will be necessary later.

On the application creation screen set the `Redirect URLs field`. This is where your application goes after a success or failure to log in to Spotify.
Set the `Redirect URis field` to `localhost:8888`.

In [0]:
cid ="682502831efa4a3992fc25308b2f8ff2" 
secret = "1b8ee7be7dd541b7bc4c45f6189c6dca"
#username = "128062522"
username = "aarondelgiudice@gmail.com"

credentials = oauth2.SpotifyClientCredentials(
    client_id=cid,
    client_secret=secret)

token = credentials.get_access_token()

if token:
  sp = spotipy.Spotify(auth=token)
else:
  print("Can't get token for", username)

In [0]:
# test that this worked
lz_uri = 'spotify:artist:36QJpDe2go2KgaRleHCDTp'

results = sp.artist_top_tracks(lz_uri)

for track in results['tracks'][:10]:
    print('track    : ' + track['name'])
    print('audio    : ' + track['preview_url'])
    print('cover art: ' + track['album']['images'][0]['url'])
    print()

In [0]:
name = 'Radiohead'

results = sp.search(q='artist:' + name, type='artist')
items = results['artists']['items']
if len(items) > 0:
    artist = items[0]
    print(artist['name'], artist['images'][0]['url'])

### Get Spotify Data

In [0]:
# get list of good tracks from playlist
good_playlist = sp.user_playlist(user="1287242681",
                                 playlist_id="5OdH7PmotfAO7qDGxKdw3J")
good_tracks = good_playlist["tracks"]
good_songs = good_tracks["items"] 
while good_tracks['next']:
    good_tracks = sp.next(good_tracks)
    for item in good_tracks["items"]:
        good_songs.append(item)
good_ids = [] 
for i in range(len(good_songs)- 500):
    good_ids.append(good_songs[i]['track']['id'])

In [0]:
# get list of bad tracks from playlist
bad_playlist = sp.user_playlist(user="1287242681",
                                playlist_id="3ySDAXYGUwRrp8C4ejIm9m")
bad_tracks = bad_playlist["tracks"]
bad_songs = bad_tracks["items"] 
while bad_tracks['next']:
    bad_tracks = sp.next(bad_tracks)
    for item in bad_tracks["items"]:
        bad_songs.append(item)
bad_ids = [] 
for i in range(len(bad_songs)- 500):
    bad_ids.append(bad_songs[i]['track']['id'])

In [0]:
# get audio features from spotify
# set traget value for 'good tracks' to 1
features = []
for i in range(0,len(good_ids),50):
    audio_features = sp.audio_features(good_ids[i:i+50])
    for track in audio_features:
        features.append(track)
        features[-1]['target'] = 1

In [0]:
# get audio features from spotify
# set traget value for 'bad tracks' to 0
for i in range(0,len(bad_ids),50):
    audio_features = sp.audio_features(bad_ids[i:i+50])
    for track in audio_features:
        features.append(track)
        features[-1]['target'] = 0

In [0]:
# load data into DataFrame
df = pd.DataFrame(features)

print(df.shape)
df.head()

### More API Data

In [0]:
print(type(df['id']))
print(df['id'][:5])

In [0]:
track_info = sp.track(df["id"][0])
print(type(track_info))
for key in track_info.keys():
  print(key, ":", track_info[key])

#### Get track names

In [0]:
sp.track(df["id"][0])['name']

In [0]:
track_names = []
for i in df.index:
  track_names.append(sp.track(df["id"][i])["name"])
  
len(track_names)

df["track_names"] = track_names

#### Get artist names

In [0]:
print(type(sp.track(df["id"][0])['artists']))
print(type(sp.track(df["id"][0])['artists'][0]))
print(sp.track(df["id"][0])['artists'][0])
for key in sp.track(df["id"][0])['artists'][0].keys():
  print(key, ':', sp.track(df["id"][0])['artists'][0][key])

In [0]:
print(sp.track(df["id"][0])['artists'][0]["name"])

In [0]:
artist_names = []
for i in df.index:
  artist_names.append(sp.track(df["id"][i])['artists'][0]["name"])
  
len(artist_names)

df["artist_names"] = artist_names

#### Get album names

In [0]:
print(type(sp.track(df["id"][0])['album']))
for key in sp.track(df["id"][0])['album'].keys():
  print(key, ':', sp.track(df["id"][0])['album'][key])

In [0]:
album_names = []
for i in df.index:
  album_names.append(sp.track(df["id"][i])['album']["name"])
  
len(album_names)

df["album_names"] = album_names

#### Get release dates

In [0]:
release_dates = []
for i in df.index:
  release_dates.append(sp.track(df["id"][i])['album']["release_date"])
  
len(release_dates)

df["release_dates"] = release_dates
# convert to data time
df["release_dates"] = pd.to_datetime(df["release_dates"])

#### Get popularity score

In [0]:
popularity = []
for i in df.index:
  popularity.append(sp.track(df["id"][i])["popularity"])
  
len(popularity)

df["popularity"] = popularity

#### Get 'explicit' boolean

In [0]:
explicit = []
for i in df.index:
  explicit.append(sp.track(df["id"][i])["explicit"])
  
len(explicit)

df["explicit"] = explicit

#### Save data for future use

In [0]:
df.head()

In [0]:
# export data locally
filename = "spotify_test_data"
df.to_csv(filename + ".csv")

## Lyric Data

You'll need to sign up for a (free) account that authorizes access to [the Genius API](http://genius.com/api-clients).
The Genius account provides a `client_access_token` that is required by the package.

### Connect to API

In [0]:
# connect to API
client_access_token = "ipUeKAQOhGtN0yUOLIp7JlOcSQfG1T6MLA4WJv1fcnqoyE1GOxX_3ObCbYmkJuI7"
genius = lyricsgenius.Genius(client_access_token)
artist = genius.search_artist("Andy Shauf", max_songs=3, sort="title")
print(artist.songs)

Searching for songs by Andy Shauf...

Song 1: "Alexander All Alone"
Song 2: "All the Same"
Song 3: "Angela"

Reached user-specified song limit (3).
Done. Found 3 songs.
[('Alexander All Alone', 'Andy Shauf'), ('All the Same', 'Andy Shauf'), ('Angela', 'Andy Shauf')]


In [0]:
# test that this worked
#song = genius.search_song("To You", artist.name)
#print(song.lyrics)

In [0]:
# intitialize a dictionary of artists-songs
# keys will be artist names and values will be song titles
# this can be used with lyric_dict to find lyrics by song title or artist name    
artist_dict = dict()

for idx in df_af.index:
  # check if artist key is already in artist_dict
  if df_af["artist_names"][idx] in artist_dict:
    # if artist key is in artist_dict, append new song to values
    artist_dict[df_af["artist_names"][idx]].append(df_af["track_names"][idx])
  else:
    # if artist key is not in artist_dict, create new key-value pair
    # NOTE: new values MUST be lists, or append method will fail
    artist_dict[df_af["artist_names"][idx]] = [df_af["track_names"][idx]]

In [0]:
# check to make sure this worked
artist_dict.keys()

dict_keys(['Lorde', 'Johnnyswim', 'Oh Wonder', 'Seven Lions', 'ILLENIUM', 'Noah Kahan', 'machineheart', 'MUNA', 'ARTY', 'Galantis', 'Louis The Child', 'Dua Lipa', 'MORTEN', 'Phoebe Ryan', 'Jeremy Zucker', 'Chelsea Cutler', 'Broods', 'Jaymes Young', 'The Chainsmokers', 'Halsey', 'Ghastly', 'Hollow Coves', 'Sander van Doorn', 'Martin Garrix', 'FRENSHIP', 'Lauv', 'Lord Huron', 'Two Friends', 'Portugal. The Man', 'FLETCHER', 'Hembree', 'Paramore', 'A R I Z O N A', 'Wrabel', 'Sigrid', 'Cheat Codes', 'Sarah Close', 'Frank Walker', 'Ellen Swartz', 'Billie Eilish', 'Steve James', 'Majik', 'Ryder', 'Yoe Mase', 'Elephante', 'Cash Cash', 'Gryffin', 'Coyote Kisses', 'The Him', 'TRACES', 'Rain Man', 'Magic Man', 'Culture Code', 'Big Wild', 'Mark Johns', 'Kanye West', 'Steve Aoki', 'Felix Cartal', 'The Vamps', 'TRACE', 'Lostboycrow', 'AJR', "Rag'n'Bone Man", 'Just Chillax', 'Coucheron', 'The Brinks', 'VÉRITÉ', 'Wingtip', 'Porter Robinson', 'Lost Kings', 'Jack Garratt', 'Kitty', 'Aquilo', 'MØ', 'Jame

In [0]:
# get first song for first five artists
for artist in df_af['artist_names'].unique()[:5]:
  print(artist, ":", artist_dict[artist][0])

Lorde : Sober
Johnnyswim : You and I
Oh Wonder : Solo
Seven Lions : Freesol
ILLENIUM : Sound of Walking Away


In [0]:
# get lyric data
# initialize a dictionary to hold data
# keys will be song title and values will be lyrics
lyrics_dict = dict() 
print("Getting song lyrics...")
# first get lyric data for the first five artists
for artist in df_af['artist_names'].unique()[:5]:
  # find song with genius lyrics API
  gen_search = genius.search_song(artist_dict[artist][0], artist)
  if type(gen_search) == lyricsgenius.song.Song:
    # if search is successful, get lyrics
    lyrics_dict[artist_dict[artist][0]] = gen_search.lyrics
  else:
    # else, set lyrics to NaN
    lyrics_dict[artist_dict[artist][0]] = np.NaN
print("Finished.", len(lyrics_dict), " total song lyrics retrieved.")

Getting song lyrics...
Searching for "Sober" by Lorde...
Done.
Searching for "You and I" by Johnnyswim...
Done.
Searching for "Solo" by Oh Wonder...
Done.
Searching for "Freesol" by Seven Lions...
Done.
Searching for "Sound of Walking Away" by ILLENIUM...
Done.
Finished. 5  total song lyrics retrieved.


In [0]:
# get lyrics for first song from first five artists
for artist in df_af['artist_names'].unique()[:5]:
  print(artist, ":", artist_dict[artist][0])
  display(lyrics_dict[artist_dict[artist][0]][:200])
  print('\n')

Lorde : Sober


"[Intro]\nNight, midnight, lose my mind\nNight, midnight, lose my mind\nNight, midnight, lose my mind\n(When you get to my high, when you get to my)\nNight, midnight\n\n[Verse 1]\nOh, God, I'm clean out of air"



Johnnyswim : You and I


'[Verse 1]\nWe found love on a stormy skies\nNever needed no reason why\nDanger always was a friend of mine\nWhen two hearts on fire collide\nCall the preacher when the sparks will fly\nYou and me burning up'



Oh Wonder : Solo


"[Verse 1: Both & Josephine]\nA room filled up with faces and the air so thick I taste it\nLocking lips like they all wasted, tangled hips like it's a spaceship\nHeavy heads are racing down the hallways t"



Seven Lions : Freesol


'[Verse]\nDidn’t see a warning\nHits me in the morning\nPuts me in the moment, the moment\nDidn’t find a reason\nHits me in the evening\nHurts me when I breathe in, I breathe in\n\n[Pre-Chorus]\nFeel the way th'



ILLENIUM : Sound of Walking Away


"[Verse 1]\nI've been quiet aside you\nWhile my heart in my chest shouts\nFeeling silenced and lied to\nBut enough is enough\n\n[Pre-Chorus]\nDon't come find me, I won't be here waiting\nI so can do it alone, "

In [0]:
# get lyric data for the full list of songs
# re-instantiat lyrics_dict
#lyrics_dict = dict()
#print("Getting song lyrics...")
#for idx in df_af.index:
#  # find song with genius lyrics API
#  genius = lyricsgenius.Genius(client_access_token)
#  gen_search = genius.search_song(df_af["track_names"][idx], df_af["artist_names"][idx])
#  if type(gen_search) == lyricsgenius.song.Song:
#    # if search is successful, get lyrics
#    lyrics_dict[df_af["track_names"][idx]] = gen_search.lyrics
#  else:
#    # else, set lyrics to NaN
#    lyrics_dict[df_af["track_names"][idx]] = np.NaN
#print("Finished.", len(lyrics_dict), " total song lyrics retrieved.")

In [0]:
batches = []

for i in range(0,len(df_af),500):
  batches.append(i)
  
print(batches)

[0, 500, 1000, 1500, 2000, 2500, 3000, 3500]


In [0]:
# get lyric data in batches
# re-instantiat lyrics_dict
lyrics_dict = dict()
print("Starting Batch 1... ")
for idx in df_af.index[batches[0]:batches[0]+500]:
  # find song with genius lyrics API
  genius = lyricsgenius.Genius(client_access_token)
  gen_search = genius.search_song(df_af["track_names"][idx], df_af["artist_names"][idx])
  if type(gen_search) == lyricsgenius.song.Song:
    # if search is successful, get lyrics
    lyrics_dict[df_af["track_names"][idx]] = gen_search.lyrics
  else:
    # else, set lyrics to NaN
    lyrics_dict[df_af["track_names"][idx]] = np.NaN
  print(np.round((idx / len(df_af) * 100), 2), '% of total songs done')
print("Finished Batch 1.", len(lyrics_dict), "song lyrics retrieved.")

Starting Batch 1... 
Searching for "Sober" by Lorde...
Done.
0.0 % of total songs done
Searching for "Homemade Dynamite" by Lorde...
Done.
0.03 % of total songs done
Searching for "The Louvre" by Lorde...
Done.
0.06 % of total songs done
Searching for "Liability" by Lorde...
Done.
0.09 % of total songs done
Searching for "Hard Feelings/Loveless" by Lorde...
Done.
0.11 % of total songs done
Searching for "Sober II (Melodrama)" by Lorde...
Done.
0.14 % of total songs done
Searching for "Writer In The Dark" by Lorde...
Done.
0.17 % of total songs done
Searching for "Supercut" by Lorde...
Done.
0.2 % of total songs done
Searching for "Liability (Reprise)" by Lorde...
Done.
0.23 % of total songs done
Searching for "Perfect Places" by Lorde...
Done.
0.26 % of total songs done
Searching for "You and I" by Johnnyswim...
Done.
0.29 % of total songs done
Searching for "Solo" by Oh Wonder...
Done.
0.31 % of total songs done
Searching for "Ultralife" by Oh Wonder...
Done.
0.34 % of total songs don

In [0]:
# get lyric data in batches
print("Starting Batch 2... ")
for idx in df_af.index[batches[1]:batches[1]+500]:
  # find song with genius lyrics API
  genius = lyricsgenius.Genius(client_access_token)
  gen_search = genius.search_song(df_af["track_names"][idx], df_af["artist_names"][idx])
  if type(gen_search) == lyricsgenius.song.Song:
    # if search is successful, get lyrics
    lyrics_dict[df_af["track_names"][idx]] = gen_search.lyrics
  else:
    # else, set lyrics to NaN
    lyrics_dict[df_af["track_names"][idx]] = np.NaN
  print(np.round((idx / len(df_af) * 100), 2), '% of total songs done')
print("Finished Batch 2.", len(lyrics_dict), "total song lyrics retrieved.")

Starting Batch 2... 
Searching for "Sound Of Your Heart" by Shawn Hook...
Done.
14.28 % of total songs done
Searching for "Idle Hands" by EXGF...
Done.
14.31 % of total songs done
Searching for "To the Top" by Twin Shadow...
Done.
14.33 % of total songs done
Searching for "Along The Road" by Karmin...
Done.
14.36 % of total songs done
Searching for "Forget And Not Slow Down" by Relient K...
Done.
14.39 % of total songs done
Searching for "Be My Escape" by Relient K...
Done.
14.42 % of total songs done
Searching for "La La Lainey" by Forever The Sickest Kids...
Done.
14.45 % of total songs done
Searching for "Parallel Lines" by ATTLAS...
Done.
14.48 % of total songs done
Searching for "Heavyweight" by RedMoon...
Done.
14.51 % of total songs done
Searching for "Disarm You (feat. Ilsey)" by Kaskade...
Done.
14.53 % of total songs done
Searching for "Borderline - Vanic Remix" by Tove Styrke...
Done.
14.56 % of total songs done
Searching for "Circles" by machineheart...
Done.
14.59 % of tot

In [0]:
# get lyric data in batches
print("Starting Batch 3... ")
for idx in df_af.index[batches[2]:batches[2]+500]:
  # find song with genius lyrics API
  genius = lyricsgenius.Genius(client_access_token)
  gen_search = genius.search_song(df_af["track_names"][idx], df_af["artist_names"][idx])
  if type(gen_search) == lyricsgenius.song.Song:
    # if search is successful, get lyrics
    lyrics_dict[df_af["track_names"][idx]] = gen_search.lyrics
  else:
    # else, set lyrics to NaN
    lyrics_dict[df_af["track_names"][idx]] = np.NaN
  print(np.round((idx / len(df_af) * 100), 2), '% of total songs done')
print("Finished Batch 3.", len(lyrics_dict), "total song lyrics retrieved.")

Starting Batch 3... 
Searching for "Little Brother" by Ella Vos...
Done.
28.56 % of total songs done
Searching for "Thru" by Vallis Alps...
Done.
28.58 % of total songs done
Searching for "Neverland" by Prismo...
Done.
28.61 % of total songs done
Searching for "Not Alone" by Otto Knows...
Done.
28.64 % of total songs done
Searching for "Gravity" by Shapeshifter...
Done.
28.67 % of total songs done
Searching for "Pieces" by CHIMES...
Done.
28.7 % of total songs done
Searching for "Haunt" by Echos...
Done.
28.73 % of total songs done
Searching for "Direction" by Hugh...
Done.
28.75 % of total songs done
Searching for "White Noise (R3hab Remix)" by Ella Vos...
Done.
28.78 % of total songs done
Searching for "Kill For Me" by Madeaux...
Done.
28.81 % of total songs done
Searching for "Not Alone" by Otto Knows...
Done.
28.84 % of total songs done
Searching for "Rift (feat. Jenn Lucas) - Original Mix" by Singularity...
No results found for: 'Rift (feat. Jenn Lucas) - Original Mix Singularity'

In [0]:
# get lyric data in batches
print("Starting Batch 4... ")
for idx in df_af.index[batches[3]:batches[3]+500]:
  # find song with genius lyrics API
  genius = lyricsgenius.Genius(client_access_token)
  gen_search = genius.search_song(df_af["track_names"][idx], df_af["artist_names"][idx])
  if type(gen_search) == lyricsgenius.song.Song:
    # if search is successful, get lyrics
    lyrics_dict[df_af["track_names"][idx]] = gen_search.lyrics
  else:
    # else, set lyrics to NaN
    lyrics_dict[df_af["track_names"][idx]] = np.NaN
  print(np.round((idx / len(df_af) * 100), 2), '% of total songs done')
print("Finished Batch 4.", len(lyrics_dict), "total song lyrics retrieved.")

Starting Batch 4... 
Searching for "Ritual" by Adam Jensen...
Done.
42.83 % of total songs done
Searching for "Line It Up" by Stephen...
Done.
42.86 % of total songs done
Searching for "The Farewell Courtyard" by FEYNMAN...
No results found for: 'The Farewell Courtyard FEYNMAN'
42.89 % of total songs done
Searching for "Ascent" by Faux Tales...
Done.
42.92 % of total songs done
Searching for "Your Light" by Yoe Mase...
Done.
42.95 % of total songs done
Searching for "Illuminate" by WILDES...
Done.
42.98 % of total songs done
Searching for "Sensations - Whethan Remix" by Elohim...
No results found for: 'Sensations - Whethan Remix Elohim'
43.0 % of total songs done
Searching for "Ammunition" by Krewella...
Done.
43.03 % of total songs done
Searching for "Better" by Mija...
Done.
43.06 % of total songs done
Searching for "Time to Say Goodbye" by Mr FijiWiji...
Done.
43.09 % of total songs done
Searching for "Who R U? (feat. Notelle)" by PatrickReza...
Done.
43.12 % of total songs done
Sea

KeyboardInterrupt: ignored

In [0]:
# get lyric data in batches
print("Starting Batch 5... ")
for idx in df_af.index[batches[4]:batches[4]+500]:
  # find song with genius lyrics API
  genius = lyricsgenius.Genius(client_access_token)
  gen_search = genius.search_song(df_af["track_names"][idx], df_af["artist_names"][idx])
  if type(gen_search) == lyricsgenius.song.Song:
    # if search is successful, get lyrics
    lyrics_dict[df_af["track_names"][idx]] = gen_search.lyrics
  else:
    # else, set lyrics to NaN
    lyrics_dict[df_af["track_names"][idx]] = np.NaN
  print(np.round((idx / len(df_af) * 100), 2), '% of total songs done')
print("Finished Batch 5.", len(lyrics_dict), "total song lyrics retrieved.")

Starting Batch 5... 
Searching for "Hi-fi Love" by Adam Carroll...
Done.
57.11 % of total songs done
Searching for "Brush up Against You" by North Mississippi Allstars...
Done.
57.14 % of total songs done
Searching for "Barbary Coast (Later)" by Conor Oberst...
Done.
57.17 % of total songs done
Searching for "L.A. Freeway" by Jerry Jeff Walker...
Done.
57.2 % of total songs done
Searching for "Days/This Time Tomorrow" by Ray Davies...
Done.
57.22 % of total songs done
Searching for "The Muse" by The Wood Brothers...
Done.
57.25 % of total songs done
Searching for "The Concept" by Teenage Fanclub...
Done.
57.28 % of total songs done
Searching for "Poor, but Proud" by Johnny Dowd...
Done.
57.31 % of total songs done
Searching for "Build Me Up From Bones" by Sarah Jarosz...
Done.
57.34 % of total songs done
Searching for "Cane River Blues" by Greg Trooper...
Done.
57.37 % of total songs done
Searching for "The Swimming Song" by Loudon Wainwright III...
Done.
57.4 % of total songs done
Sea

In [0]:
# get lyric data in batches
print("Starting Batch 6... ")
for idx in df_af.index[batches[5]:batches[5]+500]:
  # find song with genius lyrics API
  genius = lyricsgenius.Genius(client_access_token)
  gen_search = genius.search_song(df_af["track_names"][idx], df_af["artist_names"][idx])
  if type(gen_search) == lyricsgenius.song.Song:
    # if search is successful, get lyrics
    lyrics_dict[df_af["track_names"][idx]] = gen_search.lyrics
  else:
    # else, set lyrics to NaN
    lyrics_dict[df_af["track_names"][idx]] = np.NaN
  print(np.round((idx / len(df_af) * 100), 2), '% of total songs done')
print("Finished Batch 6.", len(lyrics_dict), "total song lyrics retrieved.")

Starting Batch 6... 
Searching for "Midnight Delight" by South Mountain...
Done.
71.39 % of total songs done
Searching for "The Lumberjack" by Hal Willis...
No results found for: 'The Lumberjack Hal Willis'
71.42 % of total songs done
Searching for "Boom Clap" by Lennon & Maisy...
Done.
71.44 % of total songs done
Searching for "L'alcool au volant" by Cayouche...
Done.
71.47 % of total songs done
Searching for "Heartaches By The Number" by The Canadian Sweethearts...
Done.
71.5 % of total songs done
Searching for "Dynamite Woman" by Ray St. Germain...
Done.
71.53 % of total songs done
Searching for "Don't Fence Me In" by The Cameron Family Singers...
No results found for: 'Don't Fence Me In The Cameron Family Singers'
71.56 % of total songs done
Searching for "Nothin' Like The Summer" by Carmen Rasmusen...
Done.
71.59 % of total songs done
Searching for "Dry Creek Goose Camp" by Jess Lee...
Done.
71.62 % of total songs done
Searching for "Here We Go Again" by Catalina...
Done.
71.64 % 

In [0]:
# get lyric data in batches
print("Starting Batch 7... ")
for idx in df_af.index[batches[6]:batches[6]+500]:
  # find song with genius lyrics API
  genius = lyricsgenius.Genius(client_access_token)
  gen_search = genius.search_song(df_af["track_names"][idx], df_af["artist_names"][idx])
  if type(gen_search) == lyricsgenius.song.Song:
    # if search is successful, get lyrics
    lyrics_dict[df_af["track_names"][idx]] = gen_search.lyrics
  else:
    # else, set lyrics to NaN
    lyrics_dict[df_af["track_names"][idx]] = np.NaN
  print(np.round((idx / len(df_af) * 100), 2), '% of total songs done')
print("Finished Batch 7.", len(lyrics_dict), "total song lyrics retrieved.")

Starting Batch 7... 
Searching for "Dance With My Father" by Buddy Jewel...
Done.
85.67 % of total songs done
Searching for "Daddy Can You See Me" by Anita Cochran...
Done.
85.69 % of total songs done
Searching for "Little by Little" by Clare Bowen...
Done.
85.72 % of total songs done
Searching for "Kissin' In Cars" by Jesse Lee...
Done.
85.75 % of total songs done
Searching for "Hard Drive Home" by Bailey Bryan...
Done.
85.78 % of total songs done
Searching for "Getting Even (Acoustic Version)" by Austin Webb...
No results found for: 'Getting Even (Acoustic Version) Austin Webb'
85.81 % of total songs done
Searching for "But I Can't Let You Go" by Meredith Edwards...
Done.
85.84 % of total songs done
Searching for "In Time" by Mark Collie...
Done.
85.87 % of total songs done
Searching for "I Love Nascar" by Cledus T. Judd...
Done.
85.89 % of total songs done
Searching for "The Devil Went Down to Georgia" by The Charlie Daniels Band...
Done.
85.92 % of total songs done
Searching for "A

In [0]:
# get lyric data in batches
print("Starting Batch 8... ")
for idx in df_af.index[batches[7]:batches[7]+500]:
  # find song with genius lyrics API
  genius = lyricsgenius.Genius(client_access_token)
  gen_search = genius.search_song(df_af["track_names"][idx], df_af["artist_names"][idx])
  if type(gen_search) == lyricsgenius.song.Song:
    # if search is successful, get lyrics
    lyrics_dict[df_af["track_names"][idx]] = gen_search.lyrics
  else:
    # else, set lyrics to NaN
    lyrics_dict[df_af["track_names"][idx]] = np.NaN
  print(np.round((idx / len(df_af) * 100), 2), '% of total songs done')
print("Finished Batch 8.", len(lyrics_dict), "total song lyrics retrieved.")

Starting Batch 8... 
Searching for "In Case You Didn't Know" by Brett Young...
Done.
99.94 % of total songs done
Searching for "All the Pretty Girls" by Kenny Chesney...
Done.
99.97 % of total songs done
Finished Batch 8. 2894 total song lyrics retrieved.


### Store data

In [0]:
# convert dictionary to dataframe
#lyrics_df = pd.DataFrame.from_dict(lyrics_dict, orient="index")
# create song column
#lyrics_df['songs'] = lyrics_df.index
# lable lyrics column
#lyrics_df['lyrics'] = lyrics_df[0]
# drop unused column
#lyrics_df = lyrics_df.drop(0, axis=1)
# reset index
#lyrics_df.index = range(0, len(lyrics_df))
#lyrics_df.head()

In [0]:
# store lyric data in a DataFrame
# instantiate a DF with 'artist_names', 'track_names', and 'lyrics' columns
df_lyrics = pd.DataFrame(columns=['id','artist_names', 'track_names', 'lyrics'])  
# iterate over spotify df
for idx in df_af.index:
  # for every row in spotify df, create a row in df_lyrics with artist and song title
  df_lyrics = df_lyrics.append(df_af[['id','artist_names', 'track_names']].iloc[idx])
  # add lyric data from lyric_dict using track_names as key
  try:
    df_lyrics['lyrics'].iloc[idx] = lyrics_dict[df_af['track_names'][idx]]
  except:
    # if there is no lyric data, set to NaN
    df_lyrics['lyrics'].iloc[idx] = np.NaN
# set target vale to match spotify df
df_lyrics['target'] = df_af['target']
df_lyrics.head()

,artist_names,track_names,lyrics,target
0,Lorde,Sober,"[Intro]\nNight, midnight, lose my mind\nNight,...",1
1,Lorde,Homemade Dynamite,[Verse 1]\nA couple rebel top gun pilots\nFlyi...,1
2,Lorde,The Louvre,"[Verse 1]\nWell, summer slipped us underneath ...",1
3,Lorde,Liability,"[Intro]\nOne, two\n\n[Verse 1]\nBaby really hu...",1
4,Lorde,Hard Feelings/Loveless,Hard Feelings\n\n(Go back and tell it)\n\nPlea...,1


### Check for missing data

In [0]:
print(len(df_lyrics))
df_lyrics.isna().sum()

3502


artist_names       0
track_names        0
lyrics          2874
dtype: int64

In [0]:
song_master = df['track_names']
song_missing = lyrics_df['songs']
missing_df = pd.concat([song_master, song_missing], axis=1)
missing_df.head()

print(len(missing_df))
print(len(missing_df['track_names']))
print(len(missing_df['songs']))

print(missing_df['track_names'].isna().sum())
print(missing_df['songs'].isna().sum())

missing_songs = pd.unique(missing_df[['track_names', 'songs']].values.ravel())
print(type(missing_songs))
print(len(missing_songs))
print(missing_songs[:20])

In [0]:
song_master = df['track_names']
song_missing = lyrics_df['songs']
missing_df = pd.concat([song_master, song_missing], axis=1)
missing_df.head()

print(len(missing_df))
print(len(missing_df['track_names']))
print(len(missing_df['songs']))

print(missing_df['track_names'].isna().sum())
print(missing_df['songs'].isna().sum())

missing_songs = pd.unique(missing_df[['track_names', 'songs']].values.ravel())
print(type(missing_songs))
print(len(missing_songs))
print(missing_songs[:20])

3502
3502
3502
0
608
<class 'numpy.ndarray'>
2895
['Sober' 'Homemade Dynamite' 'You and I' 'The Louvre' 'Solo' 'Liability'
 'Freesol' 'Hard Feelings/Loveless' 'Sound of Walking Away'
 'Sober II (Melodrama)' 'Writer In The Dark' 'Supercut'
 'Liability (Reprise)' 'Perfect Places' 'Ultralife' 'Lifetimes'
 'High On Humans' 'All About You' 'Heavy' 'Bigger Than Love']


In [0]:
# get lyric data for missing songs
missing_dict = dict()

print("Getting song lyrics...")
for song in missing_songs:
  # find song with genius lyrics API
  genius = lyricsgenius.Genius(client_access_token)
  gen_search = genius.search_song(song)
  if gen_search == None:
    # if song does not have lyrics, set lyrics to NaN
    missing_dict[song] = 'NaN'
  else:
    # else, store lyrics
    missing_dict[song] = gen_search.lyrics
print("Finished.")

### Save data locally

In [0]:
# export data locally
filename = "lyric_test_data_raw"
df_lyrics.to_csv(filename + ".csv")
#files.download(filename)